In [12]:
%%writefile catch_datasaver.py

# Import

import os
import sys
from datetime import datetime
import time
import requests
import re

import pandas as pd

import chromedriver_autoinstaller
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException



### 저장 경로 설정

SAVE_PATH = r'C:/DA_30_classes - 2nd Season/Project/data/'



def WC_Catch(corp=str) :
    
    # headless mode

    option = webdriver.ChromeOptions()
    option.add_argument('--headless')        # Head-less 설정
    option.add_argument('--no-sandbox')
    option.add_argument('--disable-dev-shm-usage')
    option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)" + "AppleWebKit/537.36 (KHTML, like Gecko)" + "Chrome/87.0.4280.141 Safari/537.36")

    service = Service(executable_path='chromedriver.exe')
    browser = webdriver.Chrome(service=service, options=option)

    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')

    url = r"https://www.catch.co.kr/"

    browser.maximize_window()
    print(browser.get_window_size())

    browser.get(url)



    # 로그인

    from accounts import ct

    ID = ct.ID
    PW = ct.PW



    browser.find_element(By.CSS_SELECTOR, "a.login").click()

    browser.find_element(By.CSS_SELECTOR, "input#id_login").send_keys(ID)

    browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(PW)

    browser.find_element(By.CSS_SELECTOR, "input#pw_login").send_keys(Keys.ENTER)

    time.sleep(2)



    query = browser.find_element(By.CSS_SELECTOR, "input")

    query.send_keys(corp)

    time.sleep(1)

    query.send_keys(Keys.ENTER)

    time.sleep(2)



    ### 팝업 뜨면 끄기 (예외 처리)

    try : 
        browser.find_element(By.CSS_SELECTOR, 'button.today > span').click()
        
    except : 
        pass

    time.sleep(2)



    # 검색한 기업(최상위 노출) 클릭 
    browser.find_element(By.CSS_SELECTOR, "p.name > a").click()

    time.sleep(1)

    # '현직자 리뷰' 탭 클릭
    browser.find_element(By.XPATH, r'//*[@id="Contents"]/div[1]/div[1]/div[2]/ul/li[4]/a').click()

    time.sleep(1)

    # 핵심 리뷰만 조회 - 정규직/현직자가 3년 이내 작성한 리뷰
    browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > div > a').click()

    time.sleep(1)





    #########################################################################

    time.sleep(2)

    # Review Crawler

    n_btn = browser.find_element(By.CSS_SELECTOR, 'a.ico.next')

    time.sleep(1)

    good_lst = None
    bad_lst = None


    good_lst = []
    bad_lst = []


    pre_page = 0


    while True : 

        time.sleep(1)

        # 현재 페이지의 숫자
        crt_btn = browser.find_element(By.CSS_SELECTOR, 'div.pd_type2 > p > a.selected')
        crt_page = int(crt_btn.text)

        # 다음 버튼 눌러도 같은 페이지의 경우 (마지막 페이지)
        if crt_page == pre_page:
            break

        try:
            for p in range(1, 5) : 

                grv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(4)").text
                brv = browser.find_element(By.CSS_SELECTOR, f"li:nth-child({p}) > p:nth-child(5)").text

                grv_clean = grv.replace(grv[:5], "").replace('\n', ' ') 
                brv_clean = brv.replace(brv[:4], "").replace('\n', ' ') 

                good_lst.append(grv_clean)
                bad_lst.append(brv_clean)
        
        except NoSuchElementException : 
            break

        time.sleep(2)

        # WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"li:nth-child({i}) > p:nth-child(4)")))


    # 다음 페이지로 이동하기 위한 버튼을 찾기.
        try:
            # 이전페이지의 번호를 갱신
            pre_page = crt_page

            # 다음 버튼 클릭
            n_btn.click()

            # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
            time.sleep(1)

        except:
            break



    browser.close()


    # list to dataframe

    good_lst = list(set(good_lst))
    bad_lst = list(set(bad_lst))

    good_df = pd.DataFrame({"긍정적 평가" : good_lst})
    bad_df = pd.DataFrame({"부정적 평가" : bad_lst})

    pros_cons = good_df.join(bad_df)



    # dataframe to csv file

    os.makedirs(SAVE_PATH, exist_ok=True)

    # csv 파일로 저장.

    file_name = f"{corp}_reviews.csv"
    save_file_path = os.path.join(SAVE_PATH, file_name)
    
    pros_cons.to_csv(save_file_path, index=False, encoding = "utf-8")



    # 후처리

    pros_cons['긍정적 평가'] = pros_cons['긍정적 평가'].apply(lambda x: re.sub(r'\s+', ' ',re.sub(r'\n+', ' ', x)).strip())
    pros_cons['부정적 평가'] = pros_cons['부정적 평가'].apply(lambda x: re.sub(r'\s+', ' ',re.sub(r'\n+', ' ', x)).strip())



    return pros_cons
    

Overwriting catch_datasaver.py


In [1]:
from catch_datasaver import WC_Catch

In [7]:
WC_Catch('현대모비스')

{'width': 800, 'height': 600}


,긍정적 평가,부정적 평가
0,출퇴근이 자유롭다. 복지가 잘되어있으며 고용도 안정적인편인듯,물적출자를 직원몰래 강행하여 고용에 대한 불안함을 느끼게 함
1,자기계발비가 지원이 되는 편이다. 그런점이 만족스럽다..,퇴근이 늦어져서 가족들 저녁에 챙겨주는게 힘들었음 그래도 낫배드했음
2,복지도 만족스럽고 성과급과 연봉이 높음. 사내식당 음식이 맛있는 편이라 매일이 기대...,판매실적이 좋아도 직원들 성과급 어떻게든 안주려고 노력합니다.
3,수도권에 위치하며 셔틀 버스가 상당히 많아 출퇴근하기 좋다. 책임급 전까지는 연월차...,고인물들 많음 연차 될수록 일안함 조직문화가 없음 평균 근속 평균 나이가 너무 높음
4,성장가능성이 높다고 생각하고 안정적인 직장이라 생각될 만큼 복지와 혜택이 좋고 연봉...,다양한 내외부적 이슈로 조직의 불안정성. 현기차의 그늘에서 벗어나지 못함. 주먹구구...
5,"유연근무제. (늦잠자면 늦게 출근해도 괜찮음, 연구소 기준),",팀마다 천차만별인 업무강도로 인해 워라밸의 차이가 존재함. 아직도 램4GB 컴퓨터를...
6,휴가 눈치 안보고 쓸 수 잇고 횟수도 많아서 워라밸 좋음. 사람을 자르지 않는 공무...,신입 계약 초봉 계속 하락중.. 예년과 비교해 낮은 연봉
7,5시 되면 컴퓨터 꺼짐(하지만 약간 눈치보긴 함) 안정적인 직장 역삼 근무! 젊은 ...,정규직과 비정규직의 차이가 많다. 업무강도가 높아 항상 긴장해야하며 체력이 약하면 힘들다
8,"휴가 사용이 자유롭고, 사원대리는 휴가가 아주 많다.",업무강도가 높아서 처음에 적응하기 힘이들고 많은 업무를 하다보니 실수도 자주 하게 ...
9,"어디에 배치되냐에 따라 다르지만, 대개는 칼퇴 가능 재택 및 3시퇴근도 가능해서 좋...",정말 성과급을 안주려고 노력한다. 노조가 있다. 아침 돈을 애매하게 받는다(그럴거면...
